In [6]:
import datetime
import time

import pygrasp
import grasp.plot as gplot
import matplotlib.pyplot as plt
import numpy as np
import warnings
warnings.filterwarnings("ignore", message="Converting non-nanosecond precision datetime values to nanosecond precision")

%matplotlib inline

ModuleNotFoundError: No module named 'pygrasp'

In [7]:
# EJERCICIO 1: EJECUTAR GRASP PARA SIMULAR OBSERVACIONES DE UN AEROSOL ("settingsForward")
# MODIFICAR OBSERVACIONES Y INVERTIRLAS CON DISTINTOS PARAMETROS
# Ejecutar grasp y sacar por pantalla
settingsf = pygrasp.Settings.load("settings/settingsForward.yml")
settingsf.retrieval.mode = "forward"

settingsf.input.file = "geometriaSDATA/geo_aod-4_I-4_sza75.dat"
settingsf.output.segment.stream = "output/forward_sza75.dat"
settingsf.output.sdata.dump = "sdatas_simulados/sdata_sza75.dat"

resultsForward = pygrasp.run(settingsf)

NameError: name 'pygrasp' is not defined

In [ ]:
# Lista el contenido del submódulo
print(resultsForward[0, 0, 0])

In [ ]:
resultsForward[0, 0, 0].retrieval_par.parameters[0:25]

In [ ]:
print(resultsForward[0, 0, 0].retrieval_par[0,0,0].sd[0,0:22])

In [ ]:
sdT=(resultsForward[0, 0, 0].retrieval_par[0,0,0].sd[0,0:22])
radiusT=(resultsForward[0, 0, 0].retrieval_par[0,0,0].sd[0,0:22].radius)
sdF=(resultsForward[0, 0, 0].retrieval_par[0,0,0].sd[1,0:10])
radiusF=(resultsForward[0, 0, 0].retrieval_par[0,0,0].sd[1,0:10].radius)
sdC=(resultsForward[0, 0, 0].retrieval_par[0,0,0].sd[2,0:15])
radiusC=(resultsForward[0, 0, 0].retrieval_par[0,0,0].sd[2,0:15].radius)
plt.plot(radiusT,sdT,label='Total')
plt.xscale('log')
plt.ion
plt.plot(radiusF,sdF, '-.r',label='Fino')
plt.plot(radiusC,sdC, '-.g',label='Grueso')
plt.xlabel("Radio (micras)")
plt.ylabel("Aerosol VC (um^3/um^2)");
plt.legend()
tick_positions = [0.05, 0.2, 1, 5, 15]  # Posiciones en el eje x
plt.xticks(tick_positions, labels=tick_positions)  # Usar las mismas etiquetas que las posiciones
#gplot.feature_plots.plot_real_refractive_index(resultsForward[0, 0, 0])

In [ ]:
# Salvo las vaiables
wl = (resultsForward[:, :, :].retrieved.imaginary_part_of_refractive_index_spectral_dependent[0, 0, 0, :, 0].wl)

RRI1f = (resultsForward[:, :, :].retrieved.real_part_of_refractive_index_spectral_dependent[0, 0, 0, :, 0].values)
RRI2f = (resultsForward[:, :, :].retrieved.real_part_of_refractive_index_spectral_dependent[0, 0, 0, :, 1].values)
IMI1f = (resultsForward[:, :, :].retrieved.imaginary_part_of_refractive_index_spectral_dependent[0, 0, 0, :, 0].values)
IMI2f = (resultsForward[:, :, :].retrieved.imaginary_part_of_refractive_index_spectral_dependent[0, 0, 0, :, 1].values)
print(IMI1f)

In [ ]:
#Dibujo los indices imaginarios
plt.plot(wl, IMI1f,label = "Modo 1")
plt.title(f"Indices imaginarios de refraccion con la longitud de onda");
plt.xlabel("lambda")   # Establece el título del eje x
plt.ylabel("IMI")   # Establece el título del eje y
plt.ion() 
plt.plot(wl, IMI2f,label = "Modo 2")
plt.legend(loc="best");

In [ ]:
#Dibujo los indices reales
plt.plot(wl, RRI1f,label = "Modo 1")
plt.title(f"Indices reales de refraccion con la longitud de onda");
plt.xlabel("lambda")   # Establece el título del eje x
plt.ylabel("IMI")   # Establece el título del eje y
plt.ion() 
plt.plot(wl, RRI2f,label = "Modo 2")
plt.legend(loc="best");

In [ ]:
# Busco el sdata simulado para invertirlo
sdata = pygrasp.SDATA("sdatas_simulados/sdata_sza75.dat")

# Se llama a la settings de inversion
settings = pygrasp.Settings.load("settings/settings_example_sunphotometer_inversion.yml")
settings.retrieval.mode = "inversion"
settings.output.segment.stream = "output/output_sza75.dat"


# se pueden modificar las settings con los siguietnes comandos
#settings.retrieval.constraints.characteristic[1].mode[1].single_pixel.smoothness_constraints.lagrange_multiplier=10
#settings.retrieval.constraints.characteristic[2].mode[1].single_pixel.a_priori_estimates.lagrange_multiplier=[1000,1000,1000,1000]
# settings.retrieval.constraints.characteristic[2].mode[1].single_pixel.smoothness_constraints.lagrange_multiplier=1000000
#settings.retrieval.constraints.characteristic[3].mode[1].single_pixel.smoothness_constraints.lagrange_multiplier=1000
#settings.retrieval.constraints.characteristic[1].mode[1].single_pixel.smoothness_constraints.lagrange_multiplier=0.1
#settings.retrieval.constraints.characteristic[2].mode[1].single_pixel.smoothness_constraints.lagrange_multiplier=1000
#print(sdata.pixel[0].meas[3].aod[0])
#sdata.pixel[0].meas[3].aod[0] = 0.5
results = pygrasp.run(settings, sdata=sdata, print_screen=False)

residual = np.array(results[0, 0, 0].retrieval_res.resr[:])
Conver = np.array(results[0, 0, 0].retrieval_res.rest)

In [ ]:
# DIBUJO LA SIZE DISTRIBUTION
sdF=(resultsForward[0, 0, 0].retrieval_par[0,0,0].sd[0,0:22].values)
radiusF=(resultsForward[0, 0, 0].retrieval_par[0,0,0].sd[0,0:22].radius)
VCS=results[0, 0, 0].retrieved.aerosol_concentration[0,0,0,0,0].values
sds=(results[0, 0, 0].retrieved.size_distribution_triangle_bins[0,0,0,0:22,0].values)
sdS=[]
for n in range(0,len(sds)):
    sdS.append(sds[n]*VCS)

radiusS=results[0, 0, 0].retrieval_par[0,0,0].sd[0,0:22].radius
plt.plot(radiusF,sdF,label='Forward')
plt.xscale('log')
plt.ion
plt.plot(radiusS,sdS, '-.g',label='Sin ruido')
plt.xlabel("Radio (micras)")
plt.ylabel("Aerosol VC (um^3/um^2)");
plt.legend()
tick_positions = [0.05, 0.2, 1, 5, 15]  # Posiciones en el eje x
plt.xticks(tick_positions, labels=tick_positions)  # Usar las mismas etiquetas que las posiciones
print("Error relativo medidas de Radiancia=% ")
print((residual[0] * 100))
print("Error relativo medidas de AOD=% ")
print(residual[1] * 100)
print("Error de la convergencia ")
print(Conver * 100)

In [ ]:
#Salvo las variables invertidas
wl=(results[:,:,:].retrieved.imaginary_part_of_refractive_index_spectral_dependent[0,0,0,:].wl)

RRI=(results[:,:,:].retrieved.real_part_of_refractive_index_spectral_dependent[0,0,0,:].values)
IMI=(results[:,:,:].retrieved.imaginary_part_of_refractive_index_spectral_dependent[0,0,0,:].values)
print(IMI);

In [ ]:
print(resultsForward[0, 0, 0].retrieval_par[0,0,0].sd[2,0:22])

In [ ]:
#Dibujo los indices imaginarios
plt.plot(wl, IMI1f,label = "Modo 1")
plt.title(f"Indices iamginarios de refraccion con la longitud de onda");
plt.xlabel("lambda")   # Establece el título del eje x
plt.ylabel("IMI")   # Establece el título del eje y
plt.ion() 
plt.plot(wl, IMI2f,label = "Modo 2")
plt.plot(wl, IMI,label = "Invertido")
plt.legend(loc="best");

In [ ]:
#Dibujo los indices reales con el invertido
plt.plot(wl, RRI1f,label = "Modo 1")
plt.title(f"Indices reales de refraccion con la longitud de onda");
plt.xlabel("lambda")   # Establece el título del eje x
plt.ylabel("RRI")   # Establece el título del eje y
plt.ion() 
plt.plot(wl, RRI2f,label = "Modo 2")
plt.plot(wl, RRI,label = "Invertido")
plt.legend(loc="best");

In [ ]:
##############################################
# SE MODIFICAN LAS MEDIDAS DEL SDATA METIENDO ERROR ALEATORIO
# Busco el sdata y lo modifico en sdata 2
sdata = []
sdata2 = []
sdata = pygrasp.SDATA("sdatas_simulados/sdata_sza75.dat")
sdata2 = pygrasp.SDATA("sdatas_simulados/sdata_sza75.dat")

In [ ]:
#METO RUIDO DEL 5% A LAS MEDIDAS DE RADIANCIA
NBVM=sdata.pixel[0].meas[0].nbvm[1];
ale=np.random.randn(NBVM)*0.05
Rad=sdata.pixel[0].meas[0].i[:NBVM]
sdata2.pixel[0].meas[0].i[:NBVM]=np.array(sdata.pixel[0].meas[0].i[:NBVM])+ np.array(sdata.pixel[0].meas[0].i[:NBVM])*ale
NBVM=sdata.pixel[0].meas[1].nbvm[1];
ale=np.random.randn(NBVM)*0.05
Rad=sdata.pixel[0].meas[1].i[:NBVM]
sdata2.pixel[0].meas[1].i[:NBVM]=np.array(sdata.pixel[0].meas[1].i[:NBVM])+ np.array(sdata.pixel[0].meas[1].i[:NBVM])*ale
NBVM=sdata.pixel[0].meas[2].nbvm[1];
ale=np.random.randn(NBVM)*0.05
Rad=sdata.pixel[0].meas[2].i[:NBVM]
sdata2.pixel[0].meas[2].i[:NBVM]=np.array(sdata.pixel[0].meas[2].i[:NBVM])+ np.array(sdata.pixel[0].meas[2].i[:NBVM])*ale
NBVM=sdata.pixel[0].meas[3].nbvm[1];
ale=np.random.randn(NBVM)*0.05
Rad=sdata.pixel[0].meas[3].i[:NBVM]
sdata2.pixel[0].meas[3].i[:NBVM]=np.array(sdata.pixel[0].meas[3].i[:NBVM])+ np.array(sdata.pixel[0].meas[3].i[:NBVM])*ale


In [ ]:
# METO RUIDO DE 0.01 A LAS MEDIDAS DE AOD
NBVM = sdata.pixel[0].meas[0].nbvm[0]
ale = np.random.randn(NBVM) * 0.01
AOD = sdata.pixel[0].meas[0].aod[:NBVM]
sdata2.pixel[0].meas[0].aod[:NBVM] = sdata.pixel[0].meas[0].aod[:NBVM] + ale
NBVM = sdata.pixel[0].meas[1].nbvm[0]
ale = np.random.randn(NBVM) * 0.01
AOD = sdata.pixel[0].meas[1].aod[:NBVM]
sdata2.pixel[0].meas[1].aod[:NBVM] = sdata.pixel[0].meas[1].aod[:NBVM] + ale
NBVM = sdata.pixel[0].meas[2].nbvm[0]
ale = np.random.randn(NBVM) * 0.01
AOD = sdata.pixel[0].meas[2].aod[:NBVM]
sdata2.pixel[0].meas[2].aod[:NBVM] = sdata.pixel[0].meas[2].aod[:NBVM] + ale
NBVM = sdata.pixel[0].meas[3].nbvm[0]
ale = np.random.randn(NBVM) * 0.01
AOD = sdata.pixel[0].meas[3].aod[:NBVM]
sdata2.pixel[0].meas[3].aod[:NBVM] = sdata.pixel[0].meas[3].aod[:NBVM] + ale

In [ ]:
# Se llama a la settings de inversion
settings = pygrasp.Settings.load("settings/settings_example_sunphotometer_inversion.yml")
settings.output.segment.stream = "output/outputRuido_sza75.dat"
settings.retrieval.constraints.characteristic[1].mode[1].single_pixel.smoothness_constraints.lagrange_multiplier=0.1
settings.retrieval.constraints.characteristic[2].mode[1].single_pixel.smoothness_constraints.lagrange_multiplier=1000
#settings.retrieval.constraints.characteristic[3].mode[1].single_pixel.smoothness_constraints.lagrange_multiplier=10

resultsRUIDO = pygrasp.run(settings, sdata=sdata2, print_screen=True)

In [ ]:
sdF=(resultsForward[0, 0, 0].retrieval_par[0,0,0].sd[0,0:22])
radiusF=(resultsForward[0, 0, 0].retrieval_par[0,0,0].sd[0,0:22].radius)

VCS=results[0, 0, 0].retrieved.aerosol_concentration[0,0,0,0,0].values
sds=(results[0, 0, 0].retrieved.size_distribution_triangle_bins[0,0,0,0:22,0].values)
sdS=[]
for n in range(0,len(sds)):
    sdS.append(sds[n]*VCS)
radiusS=(results[0, 0, 0].retrieval_par[0,0,0].sd[0,0:22].radius)

VCR=resultsRUIDO[0, 0, 0].retrieved.aerosol_concentration[0,0,0,0,0].values
sdr=(resultsRUIDO[0, 0, 0].retrieved.size_distribution_triangle_bins[0,0,0,0:22,0].values)
sdR=[]
for n in range(0,len(sdr)):
    sdR.append(sdr[n]*VCR)
radiusR=(resultsRUIDO[0, 0, 0].retrieval_par[0,0,0].sd[0,0:22].radius)


plt.plot(radiusF,sdF,label='Forward')
plt.xscale('log')
plt.ion
plt.plot(radiusS,sdS, '-.g',label='Sin ruido')
plt.plot(radiusR,sdR, '-.r',label='Con Ruido')
plt.xlabel("Radio (micras)")
plt.ylabel("Aerosol VC (um^3/um^2)");
plt.legend()
tick_positions = [0.05, 0.2, 1, 5, 15]  # Posiciones en el eje x
plt.xticks(tick_positions, labels=tick_positions)  # Usar las mismas etiquetas que las posiciones
residual = np.array(resultsRUIDO[0, 0, 0].retrieval_res.resr[:])
Conver = np.array(resultsRUIDO[0, 0, 0].retrieval_res.rest)
print("Error relativo medidas de Radiancia=% ")
print((residual[0] * 100))
print("Error relativo medidas de AOD=% ")
print(residual[1] * 100)
print("Error de la convergencia ")
print(Conver * 100)

In [ ]:
#Salvo las variables invertidas
RRIr=(resultsRUIDO[:,:,:].retrieved.real_part_of_refractive_index_spectral_dependent[0,0,0,:].values)
IMIr=(resultsRUIDO[:,:,:].retrieved.imaginary_part_of_refractive_index_spectral_dependent[0,0,0,:].values)

In [ ]:
#Dibujo los indices imaginarios
plt.plot(wl, IMI1f,label = "Modo 1")
plt.title(f"Indices iamginarios de refraccion con la longitud de onda");
plt.xlabel("lambda")   # Establece el título del eje x
plt.ylabel("IMI")   # Establece el título del eje y
plt.ion() 
plt.plot(wl, IMI2f,label = "Modo 2")
plt.plot(wl, IMI,label = "Invertido")
plt.plot(wl, IMIr,label = "Invertido Ruido")
plt.legend(loc="best");


In [ ]:
#Dibujo los indices reales
plt.plot(wl, RRI1f,label = "Modo 1")
plt.title(f"Indices reales de refraccion con la longitud de onda");
plt.xlabel("lambda")   # Establece el título del eje x
plt.ylabel("RRI")   # Establece el título del eje y
plt.ion() 
plt.plot(wl, RRI2f,label = "Modo 2")
plt.plot(wl, RRI,label = "Invertido")
plt.plot(wl, RRIr,label = "Invertido Ruido")
plt.legend(loc="best");